# GPC Breast Cancer EMR and Tumor Registry Data Dictionary

context:

  - [ticket:382 distribute the query to the sites for the data for consented breast cancer survey patients](https://informatics.gpcnetwork.org/trac/Project/ticket/382)
  - [GPC BC 2nd Data Pull Slides](https://docs.google.com/presentation/d/1LANts9zyDNyR3uPoArU04tsrxaC2Ao3rQyRk9JRPg9c/edit)

## Preface: PyData Scientific Python Tools

See also [PyData](http://pydata.org/).

In [1]:
# python standard library
from xml.etree import ElementTree as ET
import re

In [2]:
import pandas as pd
dict(pandas=pd.__version__)

{'pandas': u'0.17.1'}

## Record-per-tumor project

Rather than try to squish diagnoses, procedures, and medications into a one-record-per-tumor format,
we're addressing them separately:
   - Medication exposures project (below)
   - all diagnoses and procedures (ticket #nnn)

But for the tumor registry data and a few other EMR fields we're using:

  - [GPC REDCap project 32: Breast Cancer Datamart](https://redcap.gpcnetwork.org/redcap_v6.11.5/index.php?pid=32)

In [3]:
def file_access():
    from pathlib import Path
    return Path('.')
cwd = file_access()

In [4]:
bc_tumor_fields = pd.read_csv((cwd / 'bc_codebook_ddict.csv').open()).set_index('field_name')

bc_tumor_fields['v_num'] = bc_tumor_fields.index.str.extract(r'v(\d{2,3})_').astype('int')
bc_tumor_fields['naaccr_item'] = bc_tumor_fields.index.str.extract(r'v\d{2,3}_(\d{3,4})_').astype('float32')

bc_tumor_fields[['form_name', 'field_type', 'field_label', 'field_note']].head()

,form_name,field_type,field_label,field_note
field_name,,,,
v00_tumorid,admin,text,TumorID,Study ID and Sequence Number source: mixed
v01_studyid,admin,text,StudyID,Study ID source: study
v02_breastsurvey,admin,dropdown,breastsurvey,Breast Cancer Survey Participant source: study
v03_medrecordconsent,admin,dropdown,medrecordconsent,Medical Record Consent Given source: study
v13_seer_site_summary,clinical,text,SEER Site Summary,source: registry


### Tumor registry variables

Most of the variables come from the tumor registry:

In [5]:
len(bc_tumor_fields[~pd.isnull(bc_tumor_fields.naaccr_item)]), len(bc_tumor_fields)

(115, 134)

The first few are:

In [6]:
bc_tumor_fields[~pd.isnull(bc_tumor_fields.naaccr_item)][[0, 1, 2, 3]].head()

,form_name,section_header,field_type,field_label
field_name,,,,
v14_0400_primary_site,clinical,NaN,text,0400 Primary Site
v15_0380_sequence_numbercentral,clinical,NaN,dropdown,0380 Sequence Number-Central
v16_0560_sequence_numberhospital,clinical,NaN,dropdown,0560 Sequence Number-Hospital
v18_0390_date_of_diagnosis,clinical,NaN,text,0390 Date of diagnosis
v19_0230_age_at_diagnosis,clinical,NaN,text,0230 Age at Diagnosis


Variables not from NAACCR are:

In [7]:
bc_tumor_fields[pd.isnull(bc_tumor_fields.naaccr_item)][[0, 1, 2, 3]]

,form_name,section_header,field_type,field_label
field_name,,,,
v00_tumorid,admin,NaN,text,TumorID
v01_studyid,admin,NaN,text,StudyID
v02_breastsurvey,admin,NaN,dropdown,breastsurvey
v03_medrecordconsent,admin,NaN,dropdown,medrecordconsent
v13_seer_site_summary,clinical,NaN,text,SEER Site Summary
v53_deceased_per_ssa,clinical,NaN,dropdown,Deceased per SSA
v116_bmi_pre_0yr,clinical,NaN,text,BMI pre 0yr
v117_bmi_pre_0yr_date,clinical,NaN,text,BMI pre 0yr date
v118_bmi_post_0yr,clinical,NaN,text,BMI post 0yr


In [8]:
# TODO: @ vis NI for deceased

In [9]:
# TODO: other vitals. and longitudinal for vitals

### NAACCR codes: check data dictionary vs. ontology on babel

Or data dictionary has standarized mappings from codes to labels:

In [10]:
def parse_choices(txt):
    return [tuple(item.split(', ', 1))
            for item in txt.split('\n')]

ddict_choices = pd.DataFrame([
        dict(choice_code=code, code_label=label,
            naaccr_item=field.naaccr_item,
            field_label=field.field_label)
        for (name, field) in bc_tumor_fields[~pd.isnull(bc_tumor_fields.naaccr_item)].iterrows()
        if not pd.isnull(field.select_choices_or_calculations)
        for (code, label) in parse_choices(field.select_choices_or_calculations)
    ],
            columns=['naaccr_item', 'field_label', 'choice_code', 'code_label'])

ddict_choices.head()

,naaccr_item,field_label,choice_code,code_label
0,380,0380 Sequence Number-Central,00,00-One primary in the patient's lifetime
1,380,0380 Sequence Number-Central,01,01-First of two or more primaries
2,380,0380 Sequence Number-Central,02,02-Second of two or more primaries
3,380,0380 Sequence Number-Central,59,59-Fifty-ninth or higher of fifty-nine or more...
4,380,0380 Sequence Number-Central,99,99-Unspecified or unknown sequence number of F...


In [11]:
# Babel DB Access
def db_access(key='BABEL_DB'):
    from os import getenv, environ
    from sqlalchemy import create_engine

    url = getenv(key)
    if not url:
        raise IOError(key)
    return create_engine(url)

babel_db = db_access()
babel_db.execute('select 1+1').fetchone()

(2,)

### Beware leading 0s: check code lengths

Earlier codebook drafts omitted leading 0s, complicating data import.

In [12]:
# NAACCR items, labels

t_item = pd.read_sql('''
-- "Description"
select "ItemNbr", "ItemName",  "SectionID", "FieldLength", "Format", "AllowValue"
from naaccr.t_item
''', babel_db)
t_item['item'] = t_item.ItemNbr.astype('int')
t_item = t_item.set_index('item')
t_item['FieldLength'] = t_item.FieldLength.astype('float64')
t_item.head()

,ItemNbr,ItemName,SectionID,FieldLength,Format,AllowValue
item,,,,,,
10,10,Record Type,9,1,None,"I, C, A, U, M, L"
20,20,Patient ID Number,9,8,"Right justified, zero filled",None
30,30,Registry Type,9,1,None,1-3
35,35,FIN Coding System,9,1,None,"1, 2, 9"
40,40,Registry ID,9,10,"Right justified, zero filled",10-digit number. Reference to EDITS table REGI...


In [13]:
# How long is each code in the REDCap data dictionary?
x = ddict_choices[['naaccr_item', 'field_label', 'choice_code']].copy()
x['code_len'] = x.choice_code.str.len()

# Any codes from REDCap whose lengths don't match the NAACCR t_item.FieldLength?
x = x.merge(t_item[['FieldLength']], left_on='naaccr_item', right_index=True, how='left')
x[x.code_len != x.FieldLength]

,naaccr_item,field_label,choice_code,code_len,FieldLength


In [14]:
# If there are, halt and catch fire.
assert len(x[x.code_len != x.FieldLength]) == 0

### Compare GPC Ontology values vs. REDCap data dictionary values

In [15]:
def gpc_std(item_keys,
            c_table_cd='GPC'):
    '''GPC hasn't standardized c_table_cd; ignore it.
    '''
    return item_keys.apply(lambda k: '\\' + c_table_cd + '\\' + k.split('\\', 4)[-1])

def path_constraint(terms,
                    c_table_cd='GPC'):
    '''Build a SQL "in (...)" constraint from a dataframe of terms.
    '''
    paths = gpc_std(terms.item_key, c_table_cd=c_table_cd)
    params = dict(('param' + str(ix), path)
                  for (ix, path) in enumerate(paths))
    expr = ', '.join('%({0})s'.format(k) for k in params.keys())
    return expr, params

Get metadata from babel for all concepts in the NAACCR ontology:

In [16]:
naaccr_relevant = pd.read_sql(r'''
  select c_hlevel, c_name, c_fullname, c_tooltip, c_basecode
  from i2b2metadata.heron_terms
  where c_fullname like '\i2b2\naaccr\S:%%' escape '@'
  order by c_fullname
''', babel_db)

len(naaccr_relevant)

24813

Clean up the code labels and extract item and section info from path, tooltip:

In [17]:
def strip_counts(label):
    return None if label is None else re.sub(r' \[[<\d].*', '', label)

[strip_counts(txt)
 for txt in ['[AN000] ANTINEOPLASTICS [2,134,661 facts',
             '07 [<10 facts]']]

['[AN000] ANTINEOPLASTICS', '07']

In [18]:
map_option = lambda f: lambda x: None if not x else f(x)

# GPC doesn't standardize c_basecode,
# but this draws on heron_terms where the c_basecode pattern is known.
naaccr_relevant['naaccr_item'] = naaccr_relevant.c_basecode.apply(
    map_option(lambda c: int(c.split('|')[1].split(':')[0]))).astype('float64')
# Again, GPC doesn't standardize tooltips, but their structure is known for heron_terms.
naaccr_relevant['section'] = naaccr_relevant.c_tooltip.apply(
    map_option(lambda tip: ''.join(tip.split(' \\ ')[1:2])))
naaccr_relevant['item_name'] = naaccr_relevant.c_tooltip.apply(
    map_option(lambda tip: ''.join(tip.split(' \\ ')[2:3])))
naaccr_relevant['code_label'] = naaccr_relevant.c_name.apply(strip_counts)
naaccr_relevant.set_value(naaccr_relevant.c_hlevel < 4, 'code_label', None)

def naaccr_choice_codes(terms):
    return [
        (None if term.c_fullname is None or term.c_hlevel < 4 else
         term.c_fullname.split('\\')[int(term.c_hlevel + 1)].split(' ', 1)[0])
        for (_, term) in terms.iterrows()]

naaccr_relevant['choice_code'] = naaccr_choice_codes(naaccr_relevant)

In [19]:
naaccr_relevant[[
        'naaccr_item', 'section', 'item_name', 'choice_code', 'code_label']].head(8)

,naaccr_item,section,item_name,choice_code,code_label
0,NaN,,,None,None
1,2220,10 Special Use,,None,None
2,NaN,,,None,None
3,759,11 Stage/Prognostic Factors,,None,None
4,759,11 Stage/Prognostic Factors,0759 SEER Summary Stage 2000,0,0 <i>In situ</i>
5,759,11 Stage/Prognostic Factors,0759 SEER Summary Stage 2000,1,1 Localized
6,759,11 Stage/Prognostic Factors,0759 SEER Summary Stage 2000,2,"2 Regional, direct extension only"
7,759,11 Stage/Prognostic Factors,0759 SEER Summary Stage 2000,3,"3 Regional, regional lymph nodes only"


## Codes in REDCap but not in the ontology on babel

In [20]:
# First let's check that we have no NAACCR variables
# in the data dictionary that are not on babel.
bc_tumor_fields[
    ~pd.isnull(bc_tumor_fields.naaccr_item) &
    ~bc_tumor_fields.naaccr_item.isin(naaccr_relevant.naaccr_item.astype('float64'))
][['field_label', 'naaccr_item']]


,field_label,naaccr_item
field_name,,


Some codes in REDCap seem to be newer codes that are perhaps not
in the version of the NAAACCR ontology used in babel.

We're OK unless/until these codes show up in submitted data.

In [21]:
check1 = ddict_choices.set_index(['naaccr_item', 'choice_code']).join(
    naaccr_relevant.set_index(
        ['naaccr_item', 'choice_code'])[['section', 'c_basecode']],
    how='left', rsuffix='_db')

check1[pd.isnull(check1.c_basecode)]

field_label  \
naaccr_item choice_code                                  
161         16                             0161 Race 2   
            17                             0161 Race 2   
162         16                             0162 Race 3   
            17                             0162 Race 3   
163         16                             0163 Race 4   
            17                             0163 Race 4   
164         16                             0164 Race 5   
            17                             0164 Race 5   
220         5                                 0220 Sex   
            6                                 0220 Sex   
560         59           0560 Sequence Number-Hospital   
            98           0560 Sequence Number-Hospital   
610         42                      0610 Class of Case   
            43                      0610 Class of Case   
            49                      0610 Class of Case   
1646        xx           1646 RX Summ--Surg Site 98-02   
3420        999                  3420 Derived AJCC-7 M   

                                                                code_label  \
naaccr_item choice_code                                                      
161         16                                             16-Asian Indian   
            17                                                17-Pakistani   
162         16                                             16-Asian Indian   
            17                                                17-Pakistani   
163         16                                             16-Asian Indian   
            17                                                17-Pakistani   
164         16                                             16-Asian Indian   
            17                                                17-Pakistani   
220         5                                    5-Transsexual* natal male   
            6                                  6-Transsexual* natal female   
560         59           59-Fifty-ninth or higher of fifty-nine or more...   
            98           98-Cervix carcinoma in situ (CIS)/CIN III* Dia...   
610         42           42-Non-staff physician or non-CoC accredited c...   
            43                    43-PATHOLOGY or other lab specimens ONLY   
            49                                   49-DEATH CERTIFICATE ONLY   
1646        xx                                      xx-site specific codes   
3420        999                                                    999-MX    

                        section c_basecode  
naaccr_item choice_code                     
161         16              NaN        NaN  
            17              NaN        NaN  
162         16              NaN        NaN  
            17              NaN        NaN  
163         16              NaN        NaN  
            17              NaN        NaN  
164         16              NaN        NaN  
            17              NaN        NaN  
220         5               NaN        NaN  
            6               NaN        NaN  
560         59              NaN        NaN  
            98              NaN        NaN  
610         42              NaN        NaN  
            43              NaN        NaN  
            49              NaN        NaN  
1646        xx              NaN        NaN  
3420        999             NaN        NaN

@@TODO: check for codes in babel but not in REDCap?

## Code in REDCap but not in the i2b2 query

The i2b2 query **BC Phase 2e** of Dec 17, 2015 has over 100 terms from NAACCR plus several others:

In [22]:
bc295 = cwd / 'bc295_query_definition.xml'

In [23]:
def item_ont(item_key):
    part = item_key.split('\\')
    return part[5] if part[4] == 'naaccr' and part[5] == 'SEER Site' else part[4]

item_ont(r'\\i2b2_Medications\i2b2\Medications\RXAUI:3257')

'Medications'

@@TODO: site-specific factors?

In [24]:
with bc295.open() as s:
    qdef = ET.parse(s).getroot()

def e2d(e, keys):
    get = lambda ty, e: None if e is None else ty(e.text)
    return [(k, get(ty, e.find(k))) for (k, ty) in keys]

#panel_cols = [('panel_number', int),
#             ('invert', int),
#             ('panel_date_from', str),
#             ('panel_timing', str)]  # categorical
item_cols = [('hlevel', int),
             ('item_name', str),
             ('item_key', str),
             #('item_icon', str),  # categorical
             ('tooltip', str),
             #('class', str),
             #('item_is_synonym', bool)
            ]
qitem = pd.DataFrame([dict(# e2d(panel, panel_cols) +
                           e2d(item, item_cols))
              # for panel in qdef.iter('panel')
              for item in qdef.iter('item')])


qitem['short_name'] = qitem.item_name.apply(strip_counts)
del qitem['item_name']
qitem = qitem.sort_values('item_key').drop_duplicates().reset_index(drop=True)
qitem['ont'] = qitem.item_key.apply(item_ont)

qitem.tail()

,hlevel,item_key,tooltip,short_name,ont
126,3,\\i2b2_naaccr\i2b2\naaccr\S:6 Hospital-Specifi...,Cancer Cases \ 06 Hospital-Specific,0746 RX Hosp--Surg Site 98-02,naaccr
127,3,\\i2b2_naaccr\i2b2\naaccr\S:6 Hospital-Specifi...,Cancer Cases \ 06 Hospital-Specific,0747 RX Hosp--Scope Reg 98-02,naaccr
128,3,\\i2b2_naaccr\i2b2\naaccr\S:6 Hospital-Specifi...,Cancer Cases \ 06 Hospital-Specific,0748 RX Hosp--Surg Oth 98-02,naaccr
129,3,\\i2b2_naaccr\i2b2\naaccr\S:6 Hospital-Specifi...,Cancer Cases \ 06 Hospital-Specific,3280 RX Hosp--Palliative Proc,naaccr
130,3,\\i2b2_naaccr\i2b2\naaccr\SEER Site\Breast\,Cancer Cases \ SEER Site Summary,Breast,SEER Site


In [25]:
q1 = qitem.groupby('item_key')[['hlevel', 'tooltip', 'short_name']].min()
q1.reset_index()[['hlevel', 'tooltip', 'short_name', 'item_key']].to_csv('q295_sheet.csv')

In [26]:
qitem.groupby('ont')[['item_key']].count()

,item_key
ont,
Demographics,5
Diagnoses,4
Medications,2
Procedures,3
SEER Site,1
Visit Details,3
naaccr,113


For the ones from NAACCR, let's pick out the item number:

In [27]:
qitem.set_value(qitem.ont == 'naaccr', 'naaccr_item', qitem.short_name.apply(lambda s: s[:4]))
qitem.naaccr_item = qitem.naaccr_item.astype('float64')

qitem[qitem.ont == 'naaccr'][['tooltip', 'short_name', 'naaccr_item']].head()

,tooltip,short_name,naaccr_item
17,Cancer Cases \ 01 Cancer Identification,0380 Sequence Number--Central,380
18,Cancer Cases \ 01 Cancer Identification,0390 Date of Diagnosis,390
19,Cancer Cases \ 01 Cancer Identification,0400 Primary Site,400
20,Cancer Cases \ 01 Cancer Identification,0410 Laterality,410
21,Cancer Cases \ 01 Cancer Identification,0440 Grade,440


The following are missing from the query!

In [28]:
bc_tumor_fields[
    ~pd.isnull(bc_tumor_fields.naaccr_item) &
    ~bc_tumor_fields.naaccr_item.isin(qitem.naaccr_item)
][['field_label', 'form_name', 'naaccr_item']]

,field_label,form_name,naaccr_item
field_name,,,


I'm looking for them in the ontology; what section are they in?

In [29]:
naaccr_relevant[naaccr_relevant.naaccr_item.isin([2830, 2840, 3270]) & pd.isnull(naaccr_relevant.choice_code)][['naaccr_item', 'section']]

,naaccr_item,section
832,2830,11 Stage/Prognostic Factors
929,2840,11 Stage/Prognostic Factors
16522,3270,15 Treatment-1st Course


## Demographics

In [30]:
bc_tumor_fields.groupby('form_name')[['field_label']].count()

,field_label
form_name,
admin,4
clinical,46
demographic,10
registry,10
test,1
treatment,63


In [31]:
bc_tumor_fields[(bc_tumor_fields.form_name == 'demographic') &
                # NAACCR fields are already done
                pd.isnull(bc_tumor_fields.naaccr_item)]
#bc2_ddict[bc2_ddict['Form Name'] == 'Demographics'][['Form Name', 'Field Label', 'Field Type']]

,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,v_num,naaccr_item
field_name,,,,,,,,,,,,,,,,,
v52_language,demographic,NaN,text,Language,NaN,source: EMR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52,NaN


demographics TODOs@@:

  - codes for gender, rase, ...
  - codes from naaccr
  - group naaccr race with EMR race?

## Vitals

In [32]:
bc2_ddict.set_value(qitem.ont == 'Visit Details', 'Form Name', 'Visit Vitals')
bc2_ddict[bc2_ddict['Form Name'] == 'Visit Vitals'][['Form Name', 'Field Label']]

NameError: name 'bc2_ddict' is not defined

@@vitals todos:
  - baseline, 1 year, two years
  - field type, validation

In [ ]:
x = pd.DataFrame(dict(form=bc2_ddict['Form Name'],
                      tooltip=qitem.tooltip,
                      name=qitem.short_name,
                     ont=qitem.ont),
                columns='ont form tooltip name'.split())
x[pd.isnull(x.form) & ~qitem.ont.isin(['Diagnoses', 'Procedures', 'Medications'])]


### @@other

In [ ]:
qitem[~qitem.ont.isin(['naaccr', 'Diagnoses', 'Procedures'])][q_nice_cols]

#### Medications: Antineoplastics, Hormones

In [51]:
qitem[qitem.ont == 'Medications']  # [q_nice_cols]

,hlevel,item_key,tooltip,short_name,ont,naaccr_item
9,2,\\i2b2_Medications\i2b2\Medications\RXAUI:3257...,Medications,[AN000] ANTINEOPLASTICS,Medications,NaN
10,2,\\i2b2_Medications\i2b2\Medications\RXAUI:3257...,Medications,[HS000] HORMONES/SYNTHETICS/MODIFIERS,Medications,NaN


In [57]:
qitem[qitem.ont == 'Medications'].short_name.apply(lambda s: s[8:])

9                   ANTINEOPLASTICS
10    HORMONES/SYNTHETICS/MODIFIERS
Name: short_name, dtype: object

In [88]:
from urllib import urlencode

class RxClass(object):
    base='https://rxnav.nlm.nih.gov/REST/rxclass/'

    def __init__(self, opener,
                ):
        def get(resource, **params):
            addr = self._addr(resource, **params)
            print "@@getting:", addr
            body = opener.open(self._addr(resource, **params)).read()
            print "@@body:", body
            return json.loads(body)
        self.get = get

    def __getattr__(self, n):
        return lambda **params: self.get(n, **params)

    @classmethod
    def _addr(cls, resource, **params):
        return '{base}{resource}.json?{params}'.format(
        base=cls.base, resource=resource, params=urlencode(params))

RxClass._addr('byId', classId='B01AA')

'https://rxnav.nlm.nih.gov/REST/rxclass/byId.json?classId=B01AA'

In [89]:
def _x():
    import urllib2
    web = urllib2.build_opener()
    return RxClass(web)
rxclass = _x()
rxclass.get('class/byId', classId='B01AA')

@@getting: https://rxnav.nlm.nih.gov/REST/rxclass/class/byId.json?classId=B01AA
@@body: {"userInput":{"classId":"B01AA"},"rxclassMinConceptList":{"rxclassMinConcept":[{"classId":"B01AA","className":"Vitamin K antagonists","classType":"ATC1-4"}]}}


{u'rxclassMinConceptList': {u'rxclassMinConcept': [{u'classId': u'B01AA',
    u'className': u'Vitamin K antagonists',
    u'classType': u'ATC1-4'}]},
 u'userInput': {u'classId': u'B01AA'}}

In [90]:

x = qitem[qitem.ont == 'Medications'].short_name.apply(lambda s: s[8:]).apply(
    lambda n: rxclass.get('class/byName', className=n))
x

@@getting: https://rxnav.nlm.nih.gov/REST/rxclass/class/byName.json?className=ANTINEOPLASTICS
@@body: {"userInput":{"className":"ANTINEOPLASTICS","classTypes":"ALL"},"rxclassMinConceptList":{"rxclassMinConcept":[{"classId":"N0000029091","className":"ANTINEOPLASTICS","classType":"VA"}]}}
@@getting: https://rxnav.nlm.nih.gov/REST/rxclass/class/byName.json?className=HORMONES%2FSYNTHETICS%2FMODIFIERS
@@body: {"userInput":{"className":"HORMONES/SYNTHETICS/MODIFIERS","classTypes":"ALL"},"rxclassMinConceptList":{"rxclassMinConcept":[{"classId":"N0000029177","className":"HORMONES/SYNTHETICS/MODIFIERS","classType":"VA"}]}}


9     {u'userInput': {u'className': u'ANTINEOPLASTIC...
10    {u'userInput': {u'className': u'HORMONES/SYNTH...
Name: short_name, dtype: object

In [91]:
x = pd.DataFrame([item
              for v in x
             for item in v['rxclassMinConceptList']['rxclassMinConcept']])
x

,classId,className,classType
0,N0000029091,ANTINEOPLASTICS,VA
1,N0000029177,HORMONES/SYNTHETICS/MODIFIERS,VA


In [95]:
#m0 = rxclass.get('classMembers',
#                 classId='N0000029091', rela='has_VAClass', relaSource='NDFRT')
m0 = rxclass.get('classMembers',
                 classId='N0000008588', relaSource='NDFRT' , rela='has_PE')

m0

@@getting: https://rxnav.nlm.nih.gov/REST/rxclass/classMembers.json?classId=N0000008588&rela=has_PE&relaSource=NDFRT
@@body: {"userInput":{"classId":"N0000008588","trans":"0","relaSource":"NDFRT","rela":"has_PE","ttys":["IN","PIN","MIN"]},"drugMemberGroup":{"drugMember":[{"minConcept":{"rxcui":"16735","name":"acetophenazine","tty":"IN"},"nodeAttr":[{"attrName":"SourceId","attrValue":"N0000147689"},{"attrName":"SourceName","attrValue":"ACETOPHENAZINE"},{"attrName":"Relation","attrValue":"DIRECT"}]},{"minConcept":{"rxcui":"89013","name":"aripiprazole","tty":"IN"},"nodeAttr":[{"attrName":"SourceId","attrValue":"N0000148786"},{"attrName":"SourceName","attrValue":"ARIPIPRAZOLE"},{"attrName":"Relation","attrValue":"DIRECT"}]},{"minConcept":{"rxcui":"2403","name":"Chlorpromazine","tty":"IN"},"nodeAttr":[{"attrName":"SourceId","attrValue":"N0000146214"},{"attrName":"SourceName","attrValue":"CHLORPROMAZINE"},{"attrName":"Relation","attrValue":"DIRECT"}]},{"minConcept":{"rxcui":"2406","name":"Ch

{u'drugMemberGroup': {u'drugMember': [{u'minConcept': {u'name': u'acetophenazine',
     u'rxcui': u'16735',
     u'tty': u'IN'},
    u'nodeAttr': [{u'attrName': u'SourceId', u'attrValue': u'N0000147689'},
     {u'attrName': u'SourceName', u'attrValue': u'ACETOPHENAZINE'},
     {u'attrName': u'Relation', u'attrValue': u'DIRECT'}]},
   {u'minConcept': {u'name': u'aripiprazole',
     u'rxcui': u'89013',
     u'tty': u'IN'},
    u'nodeAttr': [{u'attrName': u'SourceId', u'attrValue': u'N0000148786'},
     {u'attrName': u'SourceName', u'attrValue': u'ARIPIPRAZOLE'},
     {u'attrName': u'Relation', u'attrValue': u'DIRECT'}]},
   {u'minConcept': {u'name': u'Chlorpromazine',
     u'rxcui': u'2403',
     u'tty': u'IN'},
    u'nodeAttr': [{u'attrName': u'SourceId', u'attrValue': u'N0000146214'},
     {u'attrName': u'SourceName', u'attrValue': u'CHLORPROMAZINE'},
     {u'attrName': u'Relation', u'attrValue': u'DIRECT'}]},
   {u'minConcept': {u'name': u'Chlorprothixene',
     u'rxcui': u'2406',
   

In [34]:
expr, params = path_constraint(qitem[qitem.ont == 'Medications'])
pd.read_sql(r'''
select *
from i2b2metadata.gpc_terms
where c_fullname in ({expr})
limit 100
'''.format(expr=expr), babel_db, params=params)

,c_hlevel,c_fullname,c_name,c_synonym_cd,c_visualattributes,c_totalnum,c_basecode,c_metadataxml,c_facttablecolumn,c_tablename,...,c_tooltip,m_applied_path,update_date,download_date,import_date,sourcesystem_cd,valuetype_cd,m_exclusion_cd,c_path,c_symbol
0,2,\GPC\Medications\RXAUI:3257701\,[HS000] HORMONES/SYNTHETICS/MODIFIERS,N,FA,None,RXCUI:693370,None,concept_cd,concept_dimension,...,None,@,0001-10-16 00:00:00,0001-10-16 00:00:00,0001-10-16 00:00:00,RxNORM 10052015,None,None,None,None
1,2,\GPC\Medications\RXAUI:3257528\,[AN000] ANTINEOPLASTICS,N,FA,None,RXCUI:988,None,concept_cd,concept_dimension,...,None,@,0001-10-16 00:00:00,0001-10-16 00:00:00,0001-10-16 00:00:00,RxNORM 10052015,None,None,None,None


In [35]:
# Note %s have to be doubled in sqlalchemy API
sql = r'''
with va_top as (
  select *
  from i2b2metadata.gpc_terms
  where c_fullname in ({paths})
)
, va_class as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name, sub.c_fullname
  from i2b2metadata.gpc_terms sub
  join va_top on sub.c_fullname like (va_top.c_fullname || '%%') escape '@'
  where sub.c_name like '[%%]%%'
)
select * from va_class
order by c_fullname
limit 100
'''.format(paths=expr)
med_va_class = pd.read_sql(sql, babel_db, params=params)
med_va_class['code'] = med_va_class.c_name.apply(lambda s: s.split('] ')[0][1:])
med_va_class.set_index('code')[['c_name']]

,c_name
code,
AN000,[AN000] ANTINEOPLASTICS
AN100,"[AN100] ANTINEOPLASTICS,ALKYLATING AGENTS"
AN200,[AN200] ANTINEOPLASTIC ANTIBIOTICS
AN300,"[AN300] ANTINEOPLASTICS,ANTIMETABOLITES"
AN400,[AN400] ANTINEOPLASTIC ADJUVANTS
AN500,[AN500] ANTINEOPLASTIC HORMONES
AN600,[AN600] ANTINEOPLASTIC RADIOPHARMACEUTICALS
AN700,[AN700] PROTECTIVE AGENTS
AN900,"[AN900] ANTINEOPLASTIC,OTHER"


In [99]:
# Note %s have to be doubled in sqlalchemy API
sql = r'''
with va_top as (
  select *
  from i2b2metadata.gpc_terms
  where c_fullname in ({paths})
)
, va_class as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name, sub.c_fullname
       , replace(sub.c_fullname, '\GPC\', '\i2b2\') i_fullname
  from i2b2metadata.gpc_terms sub
  join va_top on sub.c_fullname like (va_top.c_fullname || '%%') escape '@'
  where sub.c_name like '[%%]%%'
)
, rx as (
  select sub.c_hlevel, sub.c_basecode, sub.c_name
       , substr(va_class.c_name, 2, 5) va_class_code
       , sub.c_fullname
  from i2b2metadata.unmc_terms sub
  join va_class
    on sub.c_fullname between va_class.i_fullname and (va_class.i_fullname || ']')
   and sub.c_hlevel = va_class.c_hlevel + 1
  where sub.c_name not like '[%%]%%'
)
select distinct * from rx
order by va_class_code, c_basecode
'''.format(paths=expr)
rx = pd.read_sql(sql, babel_db, params=params)
print 'distinct drugs:', len(rx)
print 'hlevels:', rx.c_hlevel.unique()
rx['rxcui'] = rx.c_basecode.apply(lambda s: s.split(':')[1])

distinct drugs: 0
hlevels: []


#### Unique Drugs: eliminating polyhierarchy

In [37]:
len(rx.rxcui), len(rx.rxcui.unique())

(580, 576)

In [38]:
rx.sort_values('rxcui', inplace=True)
dup_cuis = rx[rx.rxcui.duplicated()].rxcui
rx[rx.rxcui.isin(dup_cuis)]

,c_hlevel,c_basecode,c_name,va_class_code,c_fullname,rxcui
251,4,RXCUI:372105,Ethinyl Estradiol / Norethindrone Oral Tablet,HS200,\GPC\Medications\RXAUI:3257701\RXAUI:3257706\R...,372105
530,4,RXCUI:372105,Ethinyl Estradiol / Norethindrone Oral Tablet,HS900,\GPC\Medications\RXAUI:3257701\RXAUI:3257723\R...,372105
260,4,RXCUI:374644,Mifepristone Oral Tablet,HS200,\GPC\Medications\RXAUI:3257701\RXAUI:3257706\R...,374644
538,4,RXCUI:374644,Mifepristone Oral Tablet,HS900,\GPC\Medications\RXAUI:3257701\RXAUI:3257723\R...,374644
418,5,RXCUI:377980,Glucose Oral Gel,HS502,\GPC\Medications\RXAUI:3257701\RXAUI:3257709\R...,377980
441,5,RXCUI:377980,Glucose Oral Gel,HS503,\GPC\Medications\RXAUI:3257701\RXAUI:3257709\R...,377980
160,4,RXCUI:378383,valrubicin Irrigation Solution,AN900,\GPC\Medications\RXAUI:3257528\RXAUI:3257536\R...,378383
31,4,RXCUI:378383,valrubicin Irrigation Solution,AN200,\GPC\Medications\RXAUI:3257528\RXAUI:3257530\R...,378383


In [39]:
#rx1 = pd.DataFrame(dict(va_class_code=rx.groupby('rxcui').va_class_code.min()))
rx1 = rx.groupby('rxcui')[['va_class_code', 'c_name']].min()
print len(rx1)
rx1.head()

576


,va_class_code,c_name
rxcui,,
1005923,HS200,ulipristal Oral Tablet
1043561,HS502,Metformin / saxagliptin Extended Release Oral ...
1044586,HS900,tesamorelin Injectable Solution
1045455,AN900,eribulin Injectable Solution
1090995,HS900,Ethinyl Estradiol / Norethindrone Oral Tablet ...


In [40]:
rx1['label'] = ['[{vc}] RXCUI:{cui} {drug}'.format(vc=drug.va_class_code, cui=rxcui, drug=drug.c_name)
                for (rxcui, drug) in rx1.iterrows()]
rx1.sort_values(['va_class_code', 'c_name'], inplace=True)
rx1.head(20)[['va_class_code', 'c_name', 'label']]

,va_class_code,c_name,label
rxcui,,,
377179,AN100,Busulfan Injectable Solution,[AN100] RXCUI:377179 Busulfan Injectable Solution
371165,AN100,Busulfan Oral Tablet,[AN100] RXCUI:371165 Busulfan Oral Tablet
378774,AN100,Carmustine Drug Implant,[AN100] RXCUI:378774 Carmustine Drug Implant
370496,AN100,Carmustine Injectable Solution,[AN100] RXCUI:370496 Carmustine Injectable Sol...
371375,AN100,Chlorambucil Oral Tablet,[AN100] RXCUI:371375 Chlorambucil Oral Tablet
377238,AN100,Cyclophosphamide / Mannitol Injectable Solution,[AN100] RXCUI:377238 Cyclophosphamide / Mannit...
376666,AN100,Cyclophosphamide Injectable Solution,[AN100] RXCUI:376666 Cyclophosphamide Injectab...
1437967,AN100,Cyclophosphamide Oral Capsule,[AN100] RXCUI:1437967 Cyclophosphamide Oral Ca...
371664,AN100,Cyclophosphamide Oral Tablet,[AN100] RXCUI:371664 Cyclophosphamide Oral Tablet


In [50]:
# Note %s have to be doubled in sqlalchemy API
sql = r'''
  select c_hlevel, c_basecode, c_name, c_fullname
  from i2b2metadata.gpc_terms
  where c_basecode like '%%1670010'
'''
rxx = pd.read_sql(sql, babel_db)
print 'distinct drugs:', len(rxx)
print 'hlevels:', rxx.c_hlevel.unique()
rxx['rxcui'] = rxx.c_basecode.apply(lambda s: s.split(':')[1])

distinct drugs: 0
hlevels: []


In [46]:
rxx[rxx.rxcui == '1670010']
#rxx.head()

,c_hlevel,c_basecode,c_name,c_fullname,rxcui


## Data Dictionary for REDCap

In [ ]:
ddict_prototype = pd.read_csv((cwd / 'redcap_ddict_prototype.csv').open()).set_index(u'Variable / Field Name')
ddict_prototype.columns

In [ ]:
def choices(series):
    return ' | '.join('{code}, {label}'.format(code=code, label=label)
               for (code, label) in series.iteritems())

In [ ]:
med_ddict = ddict_prototype.copy()
med_ddict.set_value('va_class', u'Choices, Calculations, OR Slider Labels', choices(med_va_class.set_index('code').c_name))
med_ddict.set_value('rxcui', u'Choices, Calculations, OR Slider Labels', choices(rx1.label))

med_ddict.to_csv('med_exposure_ddict.csv')
med_ddict